In [1]:
# IMPORTATIONS
import json
import logging
import sys
import os
from typing import List
from degiro_connector.trading.api import API as TradingAPI
from degiro_connector.trading.models.trading_pb2 import Credentials, ProductSearch, ProductsInfo
import shelve
from degiro_connector.quotecast.api import API as QuotecastAPI
from degiro_connector.quotecast.actions.action_get_chart import ChartHelper
from degiro_connector.quotecast.models.quotecast_pb2 import Chart
import pandas as pd
from datetime import datetime
import traceback
import numpy as np
from scipy.interpolate import interp1d
import re
from multiprocessing import  Pool
from functools import partial
import yfinance as yf
import threading, time, random
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import math
import itertools

class cachedApi:
    def __init__(self, file:str, credentials=Credentials):
        self.__db = shelve.open(file)
        self.__trading_api = TradingAPI(credentials=credentials)
        self.__user_token = None
        self.__quotecast_api = None
        self.mutex = threading.Lock()
        
    def logout(self):
        self.__trading_api.logout()
    
    def cache_get(self, k):
        r = None
        self.mutex.acquire()
        try:
            r = self.__db[k]
        except:
            None
        self.mutex.release()
        return r

    def cache_set(self, k,v):
        self.mutex.acquire()
        self.__db[k] = v
        self.mutex.release()
    
    def get_config(self,**kwargs):
        k = 'get_config' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_config(**kwargs)
            self.cache_set(k,r)
        self.__user_token = r['clientId']
        return r
    
    def connect(self):
        self.__trading_api.connect()
        if not self.__user_token:
            self.get_config()
        if self.__user_token:
            self.__quotecast_api = QuotecastAPI(user_token=self.__user_token)   
        #session_id = self.__trading_api.connection_storage.session_id
        #print("You are now connected, with the session id :", session_id)

    def get_products_config(self,**kwargs):
        k = 'get_products_config' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_products_config(**kwargs)
            self.cache_set(k,r)
        self.indices = {}
        for li in r['indices']:
            self.indices[li['id']] = DictObj(li)
        self.countries = {}
        for li in r['countries']:
            self.countries[li['id']] = DictObj(li)
        self.exchanges = {}
        for li in r['exchanges']:
            self.exchanges[li['id']] = DictObj(li)      
        self.stockCountries =  r['stockCountries']
        return r
     
    def get_company_ratios(self,**kwargs):
        k = 'get_company_ratios' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_company_ratios(**kwargs)
            self.cache_set(k,r)
        try:
            codes = {}
            for an in r['data']['currentRatios']['ratiosGroups']:
                for i in an['items']:
                    v = i.get('value') or np.NaN  # value
                    t = i.get('type') or None # type of parameter
                    k = i.get('id') or None # name of parameter
                    m = i.get('name') or "" # meaning
                    if t == 'N' and not pd.isna(v): v = float(v)
                    #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                    if not m.__contains__(' per '): v = v * 1#000000
                    if k:
                        codes[k] = { 'meaning':m, 'value':v }
            for i in r['data']['forecastData']['ratios']:
                #print(i)
                v = i.get('value') or np.NaN  # value
                t = i.get('type') or None # type of parameter
                k = i.get('id') or None # name of parameter
                m = i.get('name') or "" # meaning
                if t == 'N' and not pd.isna(v): v = float(v)
                #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                if not m.__contains__(' per '): v = v * 1#000000
                if k:
                    codes[k] = { 'meaning':m, 'value':v }
            for i in r['data']['consRecommendationTrend']['ratings']:
                #print(i)
                v = i.get('value') or np.NaN  # value
                k = ('ratings_'+i.get('periodType')) or None # name of parameter
                if t == 'N' and not pd.isna(v): v = float(v)
                #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                if not m.__contains__(' per '): v = v * 1#000000
                if k:
                    codes[k] = { 'meaning':'', 'value':v }
                    
            codes['priceCurrency'] = { 'meaning':'', 'value':r['data']['currentRatios']['priceCurrency'] }
        except:
            None
        return codes

    def get_financial_statements(self,**kwargs):
        k = 'get_financial_statements' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_financial_statements(**kwargs)
            self.cache_set(k,r)
        codes_array = []
        if r:
            try:
                for t in ('annual','interim'):
                    if t in r['data']:
                        for an in r['data'][t]:
                                endDate = datetime.strptime(an.get('endDate'), '%Y-%m-%d')#T%H:%M:%S')
                                fiscalYear = an.get('fiscalYear')
                                periodNumber = an.get('periodNumber') or 'Y'
                                codes = {}
                                for st in an['statements']:
                                    periodLength = st.get('periodLength')
                                    periodType = st.get('periodType')
                                    for i in st['items']:
                                        v = i.get('value') or np.NaN 
                                        if not pd.isna(v): v = float(v)
                                        if not i.get('meaning').__contains__(' per '): v = v * 1#000000
                                        codes[i.get('code')] = { 'meaning':i.get('meaning'), 'value':v }
                                codes_array += [ codes ]
            except:
                #print(k)
                #traceback.print_exc()
                #del self.cache_get(k)
                None
        return codes_array
    
    
    
    def get_estimates_summaries(self,**kwargs):
        k = 'get_estimates_summaries_' + str(kwargs)
        r = self.cache_get(k)
            #print("get_estimates_summaries cache hit", type(r))
        if r is None:
            r = self.__trading_api.get_estimates_summaries(**kwargs)
            #print("get_estimates_summaries cache miss", type(r))
            self.cache_set(k,r)
        return r
    
    def get_products_info(self,**kwargs):
        k = 'get_products_info' + str(kwargs)
        r = self.cache_get(k)
            #print("get_products_info cache hit", r)
        if r is None:
            r = self.__trading_api.get_products_info(**kwargs)
            #print("get_products_info cache miss", r)
            self.cache_set(k,r)
        return r

    def get_chart(self,**kwargs):
        k = 'get_chart' + str(kwargs)
        r = self.cache_get(k)
            #print("get_chart cache hit", r)
        if r is None:
            r = self.__quotecast_api.get_chart(**kwargs)
            #print("get_chart cache miss", r)
            self.cache_set(k,r)
        return r
   
    def product_search(self,**kwargs):
        k = 'product_search' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.product_search(**kwargs)
        if hasattr(r, 'products'):
            self.cache_set(k,r)
        else:
            r = None
        return r

    def get_company_profile(self,**kwargs):
        k = 'get_company_profile' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            #searching on Degiro
            r = self.__trading_api.get_company_profile(product_isin=kwargs['product_isin'], raw=kwargs['raw'])
            self.cache_set(k,r)
        
        codes = {}
        if r is not None and 'data' in r:
            r_data = r['data']
            try:
                codes['sector'] = r_data['sector']
            except:
                None
            try:
                codes['industry'] =  r_data['industry']
            except:
                None
            try:
                codes['country'] =  r_data['contacts']['COUNTRY']
            except:
                None
            try:
                codes['floatShares'] = float(r_data['shrFloating']) / 10**6
            except:
                None
   
            try:
                for an in r_data['ratios']['ratiosGroups']:
                    for i in an['items']:
                        v = i.get('value') or np.NaN  # value
                        t = i.get('type') or None # type of parameter
                        k = i.get('id') or None # name of parameter
                        m = i.get('name') or "" # meaning
                        if t == 'N' and not pd.isna(v): v = float(v)
                        #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                        if not m.__contains__(' per '): v = v * 1#000000
                        if k:
                            codes[k] = { 'meaning':m, 'value':v }
                for i in r_data['forecastData']['ratios']:
                    #print(i)
                    v = i.get('value') or np.NaN  # value
                    t = i.get('type') or None # type of parameter
                    k = i.get('id') or None # name of parameter
                    m = i.get('name') or "" # meaning
                    if t == 'N' and not pd.isna(v): v = float(v)
                    #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                    if not m.__contains__(' per '): v = v * 1#000000
                    if k:
                        codes[k] = { 'meaning':m, 'value':v }
            except:
                None
        else: 
            # searching on Yahoo! finance
            try:
                r = self.cache_get('Y_'+k)
            except:
                sym = yf.Ticker(kwargs['product_isin'])
                r = sym.info
                try:
                    r['marketCap'] /= 1000.0
                except:
                    pass
                self.cache_set('Y_'+k, r)
                print(f"OK from Yahoo {kwargs['product_isin']}")
            codes = r
        return codes


def isna(num):
    return num!= num    

def get(d,k):
    r = np.NaN # sys.float_info.epsilon #float("nan")np.NaN
    if d is not None and (type(d) is dict) and k in d:
        r = d[k]
        if (type(r) is dict) and ('value' in r):
            r = r['value']
    else:
        r = np.NaN
    return r

def yget(d,k):
    r = np.NaN # sys.float_info.epsilon #float("nan")np.NaN
    if d is not None and k in d:
        r=d[k]
        if (type(r) is dict) and ('value' in r):
            r = r['value']
    else:
        r = np.NaN
    try:
        r = float(r)
    except:
        r = str(r)
        if r == 'None' or r == '':
            r = np.NaN
    return r

def get_longtermprice(vwdIdSecondary:str):
    qrequest = Chart.Request()
    qrequest.culture = "fr-FR"
    qrequest.period = Chart.Interval.P10Y
    qrequest.requestid = "1"
    qrequest.resolution = Chart.Interval.P1M
    qrequest.series.append("ohlc:issueid:"+vwdIdSecondary)
    qrequest.tz = "Europe/Paris"
    chart = trading_api.get_chart(request=qrequest,raw=False)
    c2=ChartHelper.format_chart(chart=chart, copy=False)
    price = ChartHelper.serie_to_df(serie=chart.series[0])
    price["timestamp"] = pd.to_datetime(price["timestamp"], unit="s")
    price.set_index("timestamp", inplace=True)
    return price

def var2rank(X,Y,x):
    r = np.NaN #0#np.NaN
    try:
        y_interp = interp1d(x=X, y=Y,fill_value=(Y[0], Y[-1]),   bounds_error=False) #kind='linear',v
        r = float(y_interp(x))
    except:
        traceback.print_exc()
        None
    return r

def write2csv(df):
    now = datetime.now() # current date and time
    filename="degiro-export-"+now.strftime("%Y-%m-%d-%H-%M")+".csv"
    filepath='.'
    fullpath=os.path.join(filepath,filename)
    print(f"Writing csv file '{fullpath}'")
    df.to_csv(fullpath, index=True, sep=str(';'), decimal=str(','),encoding='utf-8', )



def parallelize_dataframe(df, func, n_cores=os.cpu_count()):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def assess_map(product, eee):
    p = DictObj(dict(product))
    row = {}
    #if p.isin != 'GB0007908733': return row
    try:
        company_profile = trading_api.get_company_profile(product_isin=p.isin, raw=True)
        row['symbol'] = p.symbol
        row['name'] = p.name
        row['sector'] =   yget(company_profile, 'sector')  
        row['industry'] = yget(company_profile, 'industry')
        if isinstance(row['industry'], str): row['industry'] = row['industry'].replace(' (NEC)', '')
        row['isin'] = p.isin
        row['country'] = yget(company_profile, 'country') 
        row['eee'] = 1 if row['country'] in eee else 0
        
        row['volume'] = get(company_profile,"VOL10DAVG") or yget(company_profile, 'volume')
        row['marketCap'] = get(company_profile,"MKTCAP") or yget(company_profile, 'marketCap')  
        '''
        codes = trading_api.get_financial_statements(product_isin=p.isin, raw=True) # warning: currency used in reports may be different than price currency
        row['lever'] = np.NaN
        if len(codes):
            LTLL = get(codes[0], 'LTLL') # "Total Liabilities"
            QTLE = get(codes[0], 'QTLE')# # "Total Equity";
            if not pd.isna(LTLL) and not pd.isna(QTLE) and QTLE > 0:
                row['lever'] = LTLL / QTLE
'''

        codes = trading_api.get_company_ratios(product_isin=p.isin, raw=True)
        row['Cur'] = get(codes,"priceCurrency")
        row['closePrice'] = get(codes,"NPRICE")
        #row['VOL3MAVG'] = get(codes,"VOL3MAVG")
        h = get(codes,"NHIG")
        l = get(codes,"NLOW")
        row['L%H'] = int(100*(row['closePrice'] - l)/(h-l)) if not pd.isna(h) and not pd.isna(l) and not pd.isna(row['closePrice']) and h>l else np.NaN
        
        h = get(codes,"PR1DAYPRC")
        l = get(codes,"PR5DAYPRC")
        t = get(codes,"PR13WKPCT")
        x = h-l if not pd.isna(h) and not pd.isna(l) else 0
        y = l-t if not pd.isna(l) and not pd.isna(t) else 0
        row['ΔPrice'] = max(x,y)
        row['β'] = get(codes,"BETA")
        row['Reco'] = get(codes,'ratings_CURR')
        row['ΔREV5'] = get(codes,"REVPS5YGR")  # "Revenue/share (5 yr growth)"; -- should be > 0
        if pd.isna(row['ΔREV5']):
            row['ΔREV5'] = yget(company_profile, 'revenueGrowth')
        row['ΔNPM5'] = get(codes,"NPMTRENDGR") # "Net Profit Margin growth rate, 5 year"; -- should be > 0
        row['ΔEPS'] = get(codes,'TTMEPSCHG')   # latest "Growth rate% - EPS, TTM";
        if pd.isna(row['ΔEPS']):
            row['ΔEPS'] = yget(company_profile, 'earningsGrowth')
        row['ΔEPS3'] = get(codes,"EPSGRPCT")   # "EPS Growth rate % - , 3 year CAGR";
        row['ΔEPS5'] = get(codes,"EPSTRENDGR") # "EPS growth rate %, 5 year CAGR";

        row['fPE'] = get(codes,'ProjPE')          # forward PE
        if pd.isna(row['fPE']):
            row['fPE'] = yget(company_profile, 'forwardPE')
        row['fPS'] = get(codes,'Price2ProjSales') # forward PS -- should be 2 to 4/
        row['fPEG'] = row['fPE'] / row['ΔEPS3']  if row['ΔEPS3'] and row['ΔEPS3']>0 else np.NaN    # forward PEG ratio, should be <1
        row['ROEpct'] = get(codes,'TTMROEPCT')           # Return on average equity - trailing 12 month -- should be >20%
        if pd.isna(row['ROEpct']):
            row['ROEpct'] = yget(company_profile, 'returnOnEquity')
        row['ROE5Ypct'] = get(codes,'AROE5YAVG')         # Return on average equity avg 5Y -- should be >20%
        row['P2TB'] = get(codes,'APR2TANBK')             # price to tangible book
        #row['dP2TB'] = get(codes,'BVTRENDGR')           # growth of price to tangible book, 5Y CAGR
        row['P2B'] = get(codes,'APRICE2BK')              # price to  book
        if pd.isna(row['P2B']):
            row['P2B'] = yget(company_profile, 'priceToBook')
        #row['dP2B'] = get(codes,'TanBV_AYr5CAGR')       # growth of P25B, 5Y
        row['PCF'] =  get(codes,'TTMPRCFPS')             # "Price to Cash Flow per share, near 1 idealy

        row['PE'] = get(codes,'PEINCLXOR')               #  P/E including extraordinary items - TTM - should be <50%
        if pd.isna(row['PE']):
            row['PE'] = yget(company_profile, 'trailingPE')
        row['PEG'] = row['PE'] / row['ΔEPS3'] if row['ΔEPS3'] and row['ΔEPS3']>0 else np.NaN # PEG ratio, should be <1
        if pd.isna(row['PEG']):
            row['PEG'] = yget(company_profile, 'pegRatio')
        row['PS'] = get(codes,'TTMPR2REV')               #  Price to sales - trailing 12 month  -- should be between 2 to 4
        if pd.isna(row['PS']):
            row['PS'] = yget(company_profile, 'priceToSalesTrailing12Months')
            
        row['Payout'] =  get(codes,'YLD5YAVG') 
        row['%DEBT'] =  get(codes,'QTOTD2EQ') #"Total debt/total equity, percent, should be <100%
        
        if pd.isna(row['%DEBT']):
            row['%DEBT'] = yget(company_profile, 'debtToEquity')
        row['%DEBT'] = round(row['%DEBT']) if not pd.isna(row['%DEBT']) else np.NaN
        row['BV'] =  get(codes,'QBVPS') # QTANBVPS
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'ABVPS') # ATANBVPS
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'QTANBVPS')  
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'ATANBVPS')  
        if pd.isna(row['BV']):
            row['BV'] =  yget(company_profile, 'bookValue')
        row['BV'] =  round(100 * (row['BV'] - row['closePrice']) / row['closePrice']) if not pd.isna(row['BV']) and not pd.isna(row['closePrice']) and row['BV'] > 0 and row['closePrice'] > 0 else np.NaN 
        
        # book value tangible / share price, last quarter >100% is fair

        # ratio : intrinsic value from free cash flow per share / price per share - should be > 100%
        # gain of free cash flow CAGR5Y is not available on DEGIRO, so I consider "free operational" cash flow
        dFOCF =  get(codes,'FOCF_AYr5CAGR') # gain of  free operational cash flow, CAGR 5 year.
        FCFS = get(codes,'TTMFCFSHR')   # free Cash Flow per share  - trailing 12 month
        row['IV'] = FCFS*((1-((1+dFOCF/100)*0.85)**10)/(1-((1+dFOCF/100)*0.85))+10*(((1+dFOCF/100)*0.85)**10)) if not pd.isna(dFOCF) and not pd.isna(FCFS) else np.NaN
        row['IV'] = round(100*(FCFS-row['closePrice'])/row['closePrice']) if not pd.isna(row['closePrice']) and row['closePrice']>0 and not pd.isna(row['IV']) else np.NaN
        
        try:
            # 1/ EPS
            eps = get(codes,"TTMEPSINCX")   # "EPS including extraordinary items - trailing 12 month";
            # 2/ growth rate min des 2 là, ou ΔEPS5?
            gr = min(get(codes,"REVTRENDGR"), get(codes,"TanBV_AYr5CAGR"), row['ΔEPS5'])
            # 3/ projPE ou le double du précédent, min
            ppe = min (row['fPE'], 2* gr)  
            fsv = eps*((1+gr/100)**5)*ppe/2                        
            row['FV'] = round((fsv-row['closePrice'])/row['closePrice']*100) # 0=> stock price will double in 5 years
            
        except:
            pass

        #dFOCF 	FCFS
        #BVS = get(codes,'ABVPS') #Book value (Total Equity) per share - most recent fiscal year
        #FCF = p.closePrice / get(codes,'TTMPRFCFPS') # Price to Free Cash Flow per Share - trailing 12 months" 
        #dREV3 = get(codes,"REVGRPCT") #"Growth rate% -  Revenue, 3 year";
        #dBVS5 = get(codes,"BVTRENDGR") #"Book value per share growth rate, 5 year";
        #dTBE5 = get(codes,"TanBV_AYr5CAGR") #"Tangible Book Value, Total Equity, 5 Year CAGR";
        #dCSP5 = get(codes,"CSPTRENDGR") # "Capital Spending growth rate, 5 year";      
        #row['EV/EBITD'] = EV/EBITD if EV and EBITD and EBITD>0 else 0

         #
    except:
        print(f"Error profile {p.symbol}")
        traceback.print_exc()
    return row    

def myassess(country, stock_list, info_df):
    eee = {}
    try:
        eee = {k:1 for k in pd.read_csv("eee.csv", header=None).T.values[0]}
    except:
        pass
    try:
        if hasattr(stock_list, 'products'):
            with ThreadPoolExecutor(max_workers = os.cpu_count()) as executor:
                results = executor.map(assess_map, stock_list.products, itertools.repeat(eee))
            for row in results:
                info_df = info_df.append(row,ignore_index=True)
        else:
            print("Stock market as no product", country)
        #info_df = info_df.astype({'FV':'Int64', 'IV':'Int64', 'BV' :'Int64','%DEBT' :'Int64','L%H':'Int64','eee':'Int64'})
    except:
        traceback.print_exc()
        pass
    return info_df

class DictObj:
    def __init__(self, in_dict:dict):
        assert isinstance(in_dict, dict)
        for key, val in in_dict.items():
            if isinstance(val, (list, tuple)):
               setattr(self, key, [DictObj(x) if isinstance(x, dict) else x for x in val])
            else:
               setattr(self, key, DictObj(val) if isinstance(val, dict) else val)

            
            
            
            
            
            
            
            
            

logging.basicConfig(level=logging.INFO)

username = os.getenv("GT_DG_USERNAME") or ""
password = os.getenv("GT_DG_PASSWORD") or ""

if username == "" or password == "":
    exit(0)
    
credentials = Credentials(
    int_account=None,
    username=username,
    password=password,
)


trading_api = cachedApi('/home/fab/GamestonkTerminal/.cachedb',credentials)
trading_api.connect()

try:
    # get all product list, countries, marketplaces
    products_config_dict = trading_api.get_products_config(raw=True)

    # this is the main dataframe that will be filled up
    info_df = pd.DataFrame()
    
    # stocked are browsed from counties(, and not marketplaces). This is the most reliable to get all stocks
    for li_dict in trading_api.stockCountries:
        li = DictObj(li_dict)
        stock_country_id = li.id
        country = trading_api.countries[li.country].name
        #if country != 'FR': continue
        # it's assumed that a country has less than 10x1000 stocks, so we browse up to 10 pages and stop once we got a partial page
        for page in range(0,10):
            request_stock = ProductSearch.RequestStocks(stock_country_id=stock_country_id,limit=1000,offset=page*1000,require_total=True)
            stock_list = trading_api.product_search(request=request_stock, raw=False)
            if hasattr(stock_list, 'products'):
                size = len(stock_list.products)
                print(f"country:{country} list:All ({size} stocks for page {page+1})")
                # dowload data for all stocks in the list. It's multi-thread even though the cache system is mono-thread...
                if stock_list: info_df = myassess(country, stock_list, info_df) 
                #for p in stock_list.products:
                #    assess_map(p)
                if size != 1000: break
            else:
                break
    print(f"Number of stock entries in all exchanges: {info_df.shape[0]}")
    # we remove duplicates when a stock is listed on several exchanges. ISIN code is the key.
    info_df = info_df.sort_values(by=['isin', 'volume'], ascending = False).drop_duplicates(keep = 'first', subset = 'isin')
    info_df.set_index('isin', inplace = True)
    print(f"Number of stock entries after removing duplicates: {info_df.shape[0]}")
    
        
except Exception as e:
    print(e)
    print(repr(e))
    traceback.print_exc()

try:
    trading_api.logout()
except Exception as e:
    print(e)
    print(repr(e))
    traceback.print_exc()
 

INFO:degiro_connector.trading.actions.action_connect:get_session_id:response_dict: {'isPassCodeEnabled': True, 'locale': 'fr_FR', 'redirectUrl': 'https://trader.degiro.nl/trader/', 'sessionId': 'xxx.prod_b_126_3', 'status': 0, 'statusText': 'success'}


country:FI list:All (172 stocks for page 1)
country:ES list:All (174 stocks for page 1)
country:HU list:All (31 stocks for page 1)
country:DK list:All (244 stocks for page 1)
country:PT list:All (42 stocks for page 1)
country:SE list:All (706 stocks for page 1)
country:CH list:All (235 stocks for page 1)
country:NO list:All (264 stocks for page 1)
country:NL list:All (131 stocks for page 1)
country:AT list:All (77 stocks for page 1)
country:PL list:All (411 stocks for page 1)
country:CZ list:All (38 stocks for page 1)
country:IT list:All (291 stocks for page 1)
country:SG list:All (188 stocks for page 1)
country:TR list:All (36 stocks for page 1)
country:BE list:All (152 stocks for page 1)
country:GR list:All (163 stocks for page 1)
country:FR list:All (730 stocks for page 1)
country:IE list:All (37 stocks for page 1)
country:HK list:All (784 stocks for page 1)
country:CA list:All (1000 stocks for page 1)
country:CA list:All (268 stocks for page 2)
country:GB list:All (1000 stocks for 

In [2]:
def compute_rank(info_df, tile, L, M, H):
    f=1#(1+10*tile/100)**4
    info_df['_rank_PS'] =     info_df.apply(lambda x: 0.5*(var2rank([1,2,4,5,6],[1,2*f,2*f,1.5*f,1],x['PS']) * var2rank([1,2,4,5,6],[1,2*f,2*f,1.5*f,1],x['fPS'])), axis = 1) 
    info_df['_rank_ΔREV5Y'] = info_df.apply(lambda x: var2rank([0,M['ΔREV5'],H['ΔREV5']],[0.5/f,1,2*f],x['ΔREV5']), axis = 1) 
    info_df['_rank_ΔNPM5Y'] = info_df.apply(lambda x: var2rank([0,M['ΔNPM5'],H['ΔNPM5']],[0.5/f,1,2*f],x['ΔNPM5']), axis = 1) 
    info_df['_rank_PE'] =     info_df.apply(lambda x: var2rank([L['PE'],M['PE'],H['PE']],[2*f,1,0.5/f],x['PE']) * var2rank([L['fPE'],M['fPE'],H['fPE']],[2*f,1,0.5/f],x['fPE']), axis = 1)
    info_df['_rank_PEG'] =    info_df.apply(lambda x: var2rank([0,1,1.1],[2*f,1,0.5/f],x['PEG']), axis = 1) 
    info_df['_rank_ΔEPS'] =   info_df.apply(lambda x: 0.3 * (2*var2rank([0,M['ΔEPS'],H['ΔEPS']],[0.5/f,1,2*f],x['ΔEPS']) * var2rank([0,M['ΔEPS3'],H['ΔEPS3']],[0.5/f,1,2*f],x['ΔEPS3']) * var2rank([0,M['ΔEPS5'],H['ΔEPS5']],[0.5/f,1,2*f],x['ΔEPS5'])  ), axis = 1)
    info_df['_rank_fPEG'] =   info_df.apply(lambda x: var2rank([0,1,1.1],[2*f,1,0.5/f],x['fPEG']), axis = 1) 
    info_df['_rank_ROEpct'] = info_df.apply(lambda x: var2rank([L['ROEpct'],M['ROEpct'],H['ROEpct']],[0.5/f,1,2*f],x['ROEpct']) * var2rank([L['ROE5Ypct'],M['ROE5Ypct'],H['ROE5Ypct']],[0.5/f,1,2*f],x['ROE5Ypct']), axis = 1)
    info_df['_rank_P2B'] =    info_df.apply(lambda x: var2rank([L['P2TB'],M['P2TB']],[2*f,1],x['P2TB']) * var2rank([L['P2B'],M['P2B']],[2*f,1],x['P2B']), axis = 1) 
    info_df['_rank_PCF'] =    info_df.apply(lambda x: var2rank([L['PCF'],M['PCF'],H['PCF']],[2*f,1,0.5/f],x['PCF']), axis = 1) 
    #info_df['_rank_lever'] =  info_df.apply(lambda x: var2rank([2,3],[2,0],x['lever']), axis = 1) 
    info_df['_rank_%DEBT'] =  info_df.apply(lambda x: var2rank([50,100,130],[2*f,1,0.5/f],x['%DEBT']), axis = 1) 
    #info_df['_rank_mktcap'] = info_df.apply(lambda x: var2rank([0,sys.float_info.epsilon,L['marketCap'],H['marketCap']],[0,2,0,-1],x['marketCap']), axis = 1)
    info_df['_rank_PCF'] =    info_df.apply(lambda x: var2rank([L['PCF'],M['PCF'],H['PCF']],[2*f,1,0.5/f],x['PCF']), axis = 1) 
    # mary buffet  
    #info_df['_rank_WBV10Y'] =     info_df.apply(lambda x: var2rank([L['WBV10Y'],M['WBV10Y'],H['WBV10Y']],[0,1,2],x['WBV10Y']), axis = 1)
    # intrinsic value (10 years) reported to current price - centered to 0%
    info_df['_rank_BV'] =       info_df.apply(lambda x: var2rank([L['BV'],H['BV']],[0.5/f,2*f],x['BV']), axis = 1) 
    info_df['_rank_IV'] =       info_df.apply(lambda x: var2rank([L['IV'],H['IV']],[0.5/f,2*f],x['IV']), axis = 1) 
    # fair value  - centered to 0%
    info_df['_rank_FV'] =       info_df.apply(lambda x: var2rank([L['FV'],H['FV']],[0.5/f,2*f],x['FV']), axis = 1) 
    # book value versus price per share  - centered to 0%
    rank_list = list(filter(lambda r: r.startswith('_rank_'), info_df.head()))
    info_df['_score_'+str(tile)] = info_df[rank_list].product(skipna=True,axis=1)#.fillna(0).add(tile/50)
    #info_df = info_df.drop(info_df.index[info_df['_score_'+str(tile)] < 0])
    return info_df
    
M = info_df.quantile(q=0.5).to_dict()
for tile in range(95, 94, -5):    
    L = info_df.quantile(q=1-tile/100).to_dict()
    H = info_df.quantile(q=tile/100).to_dict()
    #display(L)
    if info_df.shape[0] > os.cpu_count():
        info_df = parallelize_dataframe(info_df,partial(compute_rank, tile=tile, L=L, M=M, H=H))
    else:
        compute_rank(info_df, tile, L, M, H)

score_list = list(filter(lambda r: r.startswith('_score_'), info_df.head()))
info_df['score'] = info_df[score_list].max(axis=1)
info_df.drop(score_list, axis=1, inplace=True)
rank_list = list(filter(lambda r: r.startswith('_rank_'), info_df.head()))
info_df.drop(rank_list, axis=1, inplace=True)
info_df=info_df.sort_values(by='score', ascending=False)
#info_df.drop(['dSales5','dBook5'], axis=1, inplace=True)
#info_df.drop(['dSales5','dBook5'], axis=1, inplace=True)
#df = info_df.drop(['dSales5','dBook5'], axis=1).drop(info_df.index[info_df['score'] < 1.46]).sort_values(by=['isin','score'], ascending=False).drop_duplicates(keep='first', subset='isin').sort_values(by='score', ascending=False).set_index('isin')#.sort_values(by=['sector','score'], ascending=False)#.dropna()#.reset_index(drop=True) .head(200).
#df['score'] = pd.qcut(df['score'].rank(method='first'),q=21, retbins=False, labels=False)
write2csv(info_df)
#df=df.drop(df.index[df['score'] < 2])  
#display(info_df)

Writing csv file './degiro-export-2022-03-26-11-35.csv'


In [8]:
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.2f}'.format

split=10**(max(1,math.floor(math.log(info_df.shape[0],10))-1))
keeptop=math.floor(split*0.984)
cropsector=max(5,math.floor(split/50))
cropindustry=max(1,math.floor(cropsector/5))
print(f"Quantiles:{split}, keeping top {keeptop}th and above, limiting to {cropsector} stocks per sector and {cropindustry} per industry.")
df = info_df
df['score'] = pd.qcut(df['score'].rank(method='first'),q=split, retbins=False, labels=False)
#pd.set_option('compute.use_numba', True)
#info_df.drop(rank_list, axis=1, inplace=True)
#df = info_df.sort_values(by=['isin','score'], ascending=False).drop_duplicates(keep='first', subset='isin').sort_values(by='score', ascending=False).set_index('isin').head(200).sort_values(by=['score','sector'], ascending=False)#.dropna()#.reset_index(drop=True)
#df = info_df.drop(['dSales5','dBook5'], axis=1).drop(info_df.index[info_df['score'] < 1.46]).sort_values(by=['isin','score'], ascending=False).drop_duplicates(keep='first', subset='isin').sort_values(by=['score'], ascending=False).sort_values(by=['score'], ascending=False)# 
#df['score'] = pd.qcut(df['score'].rank(method='first'),q=21, retbins=False, labels=False)
#np.arange(0,1,0.1)
#.set_index('isin').dropna()#.reset_index(drop=True)'country',
#df = info_df.reset_index().set_index('isin').sort_values(by=['isin','score'], ascending=False).drop_duplicates(keep='first').head(100).sort_values(by=['sector','score'], ascending=False)#.dropna()#.reset_index(drop=True)
#display(df['score'])
#display(df.drop(df.index[df['score'] <18]).sort_values(by=['sector','score'], ascending=False))
#.sort_values(by=['isin','score','volume'], ascending=False).drop_duplicates(keep='first', subset='isin').sort_values(by='score', ascending=False)#.set_index('isin')#info_df .sort_values(by=['sector','score'], ascending=False)#.dropna()#.reset_index(drop=True) .head(200).
#df = df[(df['industry'].isna() | df['industry'].str.contains('Min'))]# & (df['ΔREV5'].isna() | df['ΔREV5'] > 1)
#df = df[df['isin'].str.contains('FR')]#.sort_values(by=['name'], ascending=True)#[df['FV'].notna()]
#df = df[~df['sector'].str.contains('Financial')]#.sort_values(by=['name'], ascending=True)#[df['FV'].notna()]
#df = df[~df['industry'].str.contains('Real Estate')]#.sort_values(by=['name'], ascending=True)#[df['FV'].notna()]
#df = df[~df['industry'].str.contains('REITs')]#.sort_values(by=['name'], ascending=True)#[df['FV'].notna()]
#display(df)
#df = df[df['%DEBT'] <= 150]
#df = df.drop(df.index[df['QTANBVPS'] < 10])#[df['FV'].notna()]
df=df.drop(df.index[ (df['score'] < keeptop) | (df['Reco'] > 2) | (df['sector'] == "Financial") | (df['industry'].str.contains('Real Estate'))  |(df['ΔREV5'] <6) ]) #& (df['reco'] < 2.7) | (df['FV'] <0) |(df['IV'] <-50) |(df['FV'] <-50)  |(df['BV'] <-50)|(df['eee'] <1)
#df = df.sort_values(by=['score'], ascending=False)#.head(500) .drop(info_df.index[info_df['score'] < 1.3])'sector','industry',
#df = df.sort_values(by=['sector','score'], ascending=False).groupby('sector').head(cropsector).sort_values(by=['industry','score'], ascending=False).groupby('industry').head(cropindustry).sort_values(by='score', ascending=False).sort_values(by=['score'], ascending=False)#.sort_values(by=['name'], ascending=True)
df = df.sort_values(by=['sector','score'], ascending=False).groupby('sector').head(cropsector).sort_values(by=['industry','score'], ascending=False).groupby('industry').head(cropindustry).sort_values(by='score', ascending=False).sort_values(by=['eee','score','sector','industry','BV'], ascending=False)#.sort_values(by=['name'], ascending=True)
df = df.fillna("-")
print('Please read the readme.rtf to get the meaning of all the columns.')
display(df)
write2csv(df)


Quantiles:1000, keeping top 984th and above, limiting to 20 stocks per sector and 4 per industry.
Please read the readme.rtf to get the meaning of all the columns.


,symbol,name,sector,industry,country,eee,volume,marketCap,Cur,closePrice,L%H,ΔPrice,β,Reco,ΔREV5,ΔNPM5,ΔEPS,ΔEPS3,ΔEPS5,fPE,fPS,fPEG,ROEpct,ROE5Ypct,P2TB,P2B,PCF,PE,PEG,PS,Payout,%DEBT,BV,IV,FV,score
isin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PLMRCTR00015,MRC,Mercator Medical SA,Healthcare,Medical Supplies,Poland (POL),1.00,"66,440.00","749,560,500.00",-,70.40,7.00,20.62,0.85,-,46.36,62.16,35.64,474.73,137.05,1.56,0.39,0.00,86.99,90.29,0.72,0.70,0.93,0.93,0.00,0.36,0.13,0.00,64.00,-,575.00,999
PLMRBUD00015,MRB,Mirbud SA,Capital Goods,Construction & Engineering,Poland (POL),1.00,"619,280.00","368,811,700.00",PLN,4.02,40.00,17.86,1.31,2.00,7.99,19.47,231.76,31.91,30.89,7.58,-,0.24,25.43,9.16,0.81,0.80,2.55,2.97,0.09,0.17,1.35,48.00,48.00,-34.00,109.00,999
PLKPPD000017,KPD,Koszalinskie Przedsiebiors...,Capital Goods,Wood Products,Poland (POL),1.00,630.00,"114,379,200.00",-,70.50,80.00,0.74,0.26,-,9.43,45.65,317.30,63.76,59.39,-,-,-,31.35,13.43,0.89,0.89,2.33,3.27,0.05,0.28,1.24,11.00,13.00,-79.00,-,997
DE000A2YN702,UMDK,UMT United Mobility Techno...,Technology,Mobile Application Software,Germany (DEU),1.00,"3,900.00","32,591,670.00",-,6.16,7.00,27.46,2.97,-,55.28,-,98.76,115.14,-,23.69,3.20,0.21,15.56,5.56,1.50,0.29,2.80,3.88,0.03,0.96,0.00,1.00,65.00,-,"4,025.00",996
FR0004180578,SWP,Sword Group,Technology,IT Services & Consulting,Luxembourg (LUX),1.00,"14,080.00","456,726,600.00",EUR,47.85,87.00,-6.68,0.72,1.75,8.81,26.21,374.42,144.20,38.92,28.65,2.13,0.20,50.38,22.46,6.40,3.40,6.89,7.79,0.05,2.29,5.01,11.00,-82.00,-,32.00,994
FR0000054132,ALDEL,Delfingen Industry,Technology,"Auto, Truck & Motorcycle P...",France (FRA),1.00,670.00,"118,375,700.00",EUR,44.40,15.00,6.27,1.28,2.00,6.09,39.31,"1,600.34",68.65,47.70,9.04,0.33,0.13,58.30,21.72,2.42,1.01,1.54,1.89,0.03,0.34,1.71,101.00,10.00,-101.00,237.00,994
MHY8162K2046,SBLK,Star Bulk Carriers Corp,Transportation,Deep Sea Freight,Greece (GRC),1.00,"2,307,280.00","3,025,879,000.00",USD,29.42,85.00,-0.98,0.87,1.82,24.68,-,"6,926.17",107.08,-,4.56,2.48,0.04,37.50,9.95,1.45,1.45,3.61,4.42,0.04,2.11,-,74.00,-31.00,-,4.00,992
CY0103562118,THS,Tharisa PLC,Basic Materials,Platinum Mining,Cyprus (CYP),1.00,"126,200.00","8,488,004,000.00",ZAR,29.71,66.00,2.71,1.05,1.60,20.88,25.13,129.82,26.75,47.23,4.99,0.88,0.19,25.25,15.91,1.33,1.25,3.49,5.51,0.21,0.98,3.38,8.00,-20.00,-85.00,-15.00,990
PLIZSTL00015,IZS,Izostal SA,Basic Materials,Metallic Rolling & Drawing...,Poland (POL),1.00,"84,620.00","92,992,960.00",-,2.84,33.00,5.34,0.74,-,32.57,8.58,27.14,18.43,43.94,-,-,-,8.76,6.53,0.44,0.44,3.36,4.95,0.27,0.12,2.72,33.00,139.00,-67.00,-,990


Writing csv file './degiro-export-2022-03-26-13-45.csv'
